In [1]:
import numpy as np
import pandas as pd

from KUtils.eda import chartil
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from xgboost.sklearn import XGBClassifier

from sklearn import preprocessing

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer, recall_score, precision_score
from sklearn.metrics import matthews_corrcoef

import matplotlib.pyplot as plt
import seaborn as sns

/home/kooud/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
# Show all column in head()
pd.set_option('display.max_columns', None)

In [3]:
merged_tweet_bowdf = pd.read_csv('merged_tweet_bow_df.csv')

/home/kooud/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
import pickle 

with open('bow_columns.pickle', 'rb') as filehandle:
     bow_columns = pickle.load(filehandle)
        
with open('primary_columns.pickle', 'rb') as filehandle:
     primary_columns = pickle.load(filehandle)

In [11]:
# There is one row with mixed up column

merged_tweet_bowdf = merged_tweet_bowdf.replace('5d_gain_percent', 0)
merged_tweet_bowdf = merged_tweet_bowdf.replace('5d_loss_percent', 0)
merged_tweet_bowdf = merged_tweet_bowdf.replace('volume_vs_5d_avg_vol', 0)



merged_tweet_bowdf['5d_gain_percent'] = merged_tweet_bowdf['5d_gain_percent'].astype('float64')
merged_tweet_bowdf['5d_loss_percent'] = merged_tweet_bowdf['5d_loss_percent'].astype('float64')
merged_tweet_bowdf['volume_vs_5d_avg_vol'] = merged_tweet_bowdf['volume_vs_5d_avg_vol'].astype('float64')

In [18]:
merged_tweet_bowdf.head()

tweetId           created_at lang  isRetweet  \
0  418443702579654656  2014-01-01 13:08:00   en      False   
1  418396495692316672  2014-01-01 10:01:00   en      False   
2  418198017015959552  2013-12-31 20:52:00   en      False   
3  418600422740197378  2014-01-01 23:31:00   en       True   
4  418765690913705984  2014-01-02 10:28:00   en       True   

  parent_tweet_created_at  hoursDiffFromParent  tweet_userId  followers_count  \
0     2014-01-01 13:08:00                    0      23059499            48108   
1     2014-01-01 10:01:00                    0      23669783            14958   
2     2013-12-31 20:52:00                    0      23954327            17449   
3     2013-12-30 11:13:00                   60    1653399757               21   
4     2014-01-02 08:55:00                    1    1951070833                5   

  company_name afinn_verdict  afinn_score  afinn_comparative  \
0         AAPL      POSITIVE            1           0.166667   
1         AAPL      NEGATIVE           -1          -0.142857   
2         AAPL      POSITIVE            2           0.153846   
3         AAPL      POSITIVE            1           0.066667   
4         AAPL      POSITIVE            6           0.272727   

   no_of_positive_words  no_of_negative_words positive_words negative_words  \
0                     1                     0          boost            NaN   
1                     1                     1            big        screwed   
2                     1                     0    intelligent            NaN   
3                     1                     0           huge            NaN   
4                     2                     0       wins,top            NaN   

                                        cleaned_text  \
0                    itv will boost apple aapl apple   
1                apple screwed up big time amzn aapl   
2  iphone users are more intelligent than samsung...   
3  rt stocktwits here's how apple could be making...   
4  rt philstockworld our top 3 trade ideas for 20...   

                                            raw_text                 Date  \
0  iTV Will Boost Apple http://t.co/8dup4cQc08 $A...  2013-12-31 00:00:00   
1  Apple Screwed Up Big Time http://t.co/Q2Pzk2VO...  2013-12-31 00:00:00   
2  #iPhone users are more intelligent than #Samsu...  2014-01-02 00:00:00   
3  RT @StockTwits: Here's how Apple could be maki...  2014-01-02 00:00:00   
4  RT @philstockworld: Our Top 3 Trade Ideas for ...  2014-01-02 00:00:00   

        Open       High        Low      Close  Adj Close      Volume  \
0  79.167145  80.182854  79.142860  80.145714  74.571281  55771100.0   
1  79.167145  80.182854  79.142860  80.145714  74.571281  55771100.0   
2  79.382858  79.575714  78.860001  79.018570  73.522530  58671200.0   
3  79.382858  79.575714  78.860001  79.018570  73.522530  58671200.0   
4  79.382858  79.575714  78.860001  79.018570  73.522530  58671200.0   

   volume_vs_5d_avg_vol  5d_gain_percent  5d_loss_percent 20d_gain_percent  \
0              0.824783              0.0        -3.682317                0   
1              0.824783              0.0        -3.682317                0   
2              1.092410              0.0        -3.336840         0.569091   
3              1.092410              0.0        -3.336840         0.569091   
4              1.092410              0.0        -3.336840         0.569091   

  20d_loss_percent                                   cleaned_text_new  \
0         -9.99639                          iTV Will Boost Apple AAPL   
1         -9.99639                Apple Screwed Up Big Time AMZN AAPL   
2         -10.0214  iPhone users are more intelligent than and HTC...   
3         -10.0214  Here how Apple could be making a huge push int...   
4         -10.0214  Our Top  Trade Ideas for  and Two  Futures Win...   

                                   preprocessed_text  aapl  fb  stock  goog  \
0                                itv boost appl aapl     1   0      0     

In [14]:
top_company_names = ["AAPL","ABBV","AGFS","AMGN","AMZN","BABA","BRK-A","BSAC","CELG","CHTR","CMCSA","CODI","CSCO","FB","GOOG","HSBC","INTC","MSFT", "ORCL","PCLN","PICO","RDS-B","SPLP"]

In [15]:
top_tweets = merged_tweet_bowdf.loc[merged_tweet_bowdf['company_name'].isin(top_company_names)]

In [16]:
# Remove scrip names from ifidf
scrip_names = ["AAPL","ABB","ABBV","AEP","AGFS","AMGN","AMZN","BA","BABA","BAC","BBL","BCH","BHP","BP","BRK-A","BSAC","BUD","C","CAT","CELG","CHL","CHTR","CMCSA","CODI","CSCO","CVX",
              "D","DHR","DIS","DUK","EXC","FB","GD","GE","GOOG","HD","HON","HRG","HSBC","IEP","INTC","JNJ","JPM","KO","LMT","MA","MCD","MDT","MMM","MO","MRK","MSFT","NEE","NGG",
              "NVS","ORCL","PCG","PCLN","PEP","PFE","PG","PICO","PM","PPL","PTR","RDS-B","REX","SLB","SNP","SNY","SO","SPLP","SRE","T","TM","TOT","TSM","UL","UN","UNH","UPS","UTX",
              "V","VZ","WFC","WMT","XOM"]

final_bow_columns =  [i for i in bow_columns if not i.upper() in scrip_names]


In [17]:
print(len(final_bow_columns))
print(len(bow_columns))

4928
5000


In [21]:
iteration_info = pd.DataFrame( columns = ['Group', 'Dataset', 'Filter', 'Model', 'TN', 'FP', 'FN', 'TP', 'Accuracy', 'Sensitivity', 'Specificity', 
                                                'Precision', 'Recall', 'F1', 'AUC', 'MCC', 'Long_gain_mean', 'Short_gain_mean'])

def runAll(group, dataset, filter, df):
    df['target'] = df['afinn_verdict'].map({'POSITIVE':1, 'NEGATIVE':0})
    df['target'] = df['target'].astype('category')
    feature_columns = ['5d_gain_percent', '5d_loss_percent'] + final_bow_columns


    X = df[feature_columns]
    X = X.drop('target', axis=1)
    y = df['target'] 

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=43)
    
   
    # 2. GNB
    model_name ='GNB'
    clf = GaussianNB()
    clf.fit(X_train.drop(['5d_gain_percent', '5d_loss_percent'], axis=1),y_train)
    y_pred_default = clf.predict(X_test.drop(['5d_gain_percent', '5d_loss_percent'], axis=1))
    
    conf_mat = metrics.confusion_matrix(y_test, y_pred_default)
    accuracy = metrics.accuracy_score(y_test, y_pred_default)
    precision = metrics.precision_score(y_test, y_pred_default)
    recall = metrics.recall_score(y_test, y_pred_default)
    f1_score = metrics.f1_score(y_test, y_pred_default)
    roc_auc = metrics.roc_auc_score(y_test, y_pred_default)
    sensitivity = recall
    specificity =  conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
    mcc = matthews_corrcoef(y_test, y_pred_default)
    
    X_test['prediction'] = y_pred_default

    pred_positive = X_test.loc[X_test['prediction']==1]
    pred_negative = X_test.loc[X_test['prediction']==0]

    #print('Long mean  {0:.3f}'.format(pred_positive['5d_gain_percent'].mean()))
    #print('Short mean {0:.3f}'.format(pred_negative['5d_loss_percent'].mean()))    
    del X_test['prediction']
    
    iteration_info.loc[iteration_info.shape[0]]=[group, dataset, filter, model_name, conf_mat[0][0], conf_mat[0][1], conf_mat[1][0], conf_mat[1][1],
                                                 accuracy, sensitivity, specificity, precision, recall, f1_score, roc_auc, mcc, 
                                                 pred_positive['5d_gain_percent'].mean(), pred_negative['5d_loss_percent'].mean()]
    
    # 3. MNB
    model_name ='MNB'
    clf = MultinomialNB()
    clf.fit(X_train.drop(['5d_gain_percent', '5d_loss_percent'], axis=1),y_train)
    y_pred_default = clf.predict(X_test.drop(['5d_gain_percent', '5d_loss_percent'], axis=1))
    
    conf_mat = metrics.confusion_matrix(y_test, y_pred_default)
    accuracy = metrics.accuracy_score(y_test, y_pred_default)
    precision = metrics.precision_score(y_test, y_pred_default)
    recall = metrics.recall_score(y_test, y_pred_default)
    f1_score = metrics.f1_score(y_test, y_pred_default)
    roc_auc = metrics.roc_auc_score(y_test, y_pred_default)
    sensitivity = recall
    specificity =  conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
    mcc = matthews_corrcoef(y_test, y_pred_default)
    
    X_test['prediction'] = y_pred_default

    pred_positive = X_test.loc[X_test['prediction']==1]
    pred_negative = X_test.loc[X_test['prediction']==0]

    #print('Long mean  {0:.3f}'.format(pred_positive['5d_gain_percent'].mean()))
    #print('Short mean {0:.3f}'.format(pred_negative['5d_loss_percent'].mean()))    
    del X_test['prediction']
    
    iteration_info.loc[iteration_info.shape[0]]=[group, dataset, filter, model_name, conf_mat[0][0], conf_mat[0][1], conf_mat[1][0], conf_mat[1][1],
                                                 accuracy, sensitivity, specificity, precision, recall, f1_score, roc_auc, mcc, 
                                                 pred_positive['5d_gain_percent'].mean(), pred_negative['5d_loss_percent'].mean()]
    
    # 4. BNB
    model_name ='BNB'
    clf = BernoulliNB()
    clf.fit(X_train.drop(['5d_gain_percent', '5d_loss_percent'], axis=1),y_train)
    y_pred_default = clf.predict(X_test.drop(['5d_gain_percent', '5d_loss_percent'], axis=1))
    
    conf_mat = metrics.confusion_matrix(y_test, y_pred_default)
    accuracy = metrics.accuracy_score(y_test, y_pred_default)
    precision = metrics.precision_score(y_test, y_pred_default)
    recall = metrics.recall_score(y_test, y_pred_default)
    f1_score = metrics.f1_score(y_test, y_pred_default)
    roc_auc = metrics.roc_auc_score(y_test, y_pred_default)
    sensitivity = recall
    specificity =  conf_mat[0,0]/(conf_mat[0,0]+conf_mat[0,1])
    mcc = matthews_corrcoef(y_test, y_pred_default)
    
    X_test['prediction'] = y_pred_default

    pred_positive = X_test.loc[X_test['prediction']==1]
    pred_negative = X_test.loc[X_test['prediction']==0]

    #print('Long mean  {0:.3f}'.format(pred_positive['5d_gain_percent'].mean()))
    #print('Short mean {0:.3f}'.format(pred_negative['5d_loss_percent'].mean()))    
    del X_test['prediction']
    
    iteration_info.loc[iteration_info.shape[0]]=[group, dataset, filter, model_name, conf_mat[0][0], conf_mat[0][1], conf_mat[1][0], conf_mat[1][1],
                                                 accuracy, sensitivity, specificity, precision, recall, f1_score, roc_auc, mcc, 
                                                 pred_positive['5d_gain_percent'].mean(), pred_negative['5d_loss_percent'].mean()]    
   

In [22]:
df = merged_tweet_bowdf
runAll('G1', 'Full', 'None', df)

/home/kooud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kooud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kooud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [23]:
iteration_info.head()

,Group,Dataset,Filter,Model,TN,FP,FN,TP,Accuracy,Sensitivity,Specificity,Precision,Recall,F1,AUC,MCC,Long_gain_mean,Short_gain_mean
0,G1,Full,None,GNB,3481,203,5845,3942,0.551036,0.402779,0.944897,0.951025,0.402779,0.565891,0.673838,0.335776,1.674160,-1.511531
1,G1,Full,None,MNB,3201,483,576,9211,0.921387,0.941146,0.868893,0.950175,0.941146,0.945639,0.905019,0.803832,1.566162,-1.410232
2,G1,Full,None,BNB,3191,493,589,9198,0.919679,0.939818,0.866178,0.949128,0.939818,0.944450,0.902998,0.799627,1.565084,-1.413925


In [45]:
df = merged_tweet_bowdf
runAll('G1', 'Full', 'None', df)

df = merged_tweet_bowdf.loc[(merged_tweet_bowdf['followers_count']>1000)]
runAll('G2', 'Full', 'Follower>1K', df)

df = merged_tweet_bowdf.loc[(merged_tweet_bowdf['followers_count']>1000) & (merged_tweet_bowdf['hoursDiffFromParent']<2)]
runAll('G3', 'Full', 'Freshness<2 & Follower>1K', df)

df = merged_tweet_bowdf.loc[(merged_tweet_bowdf['volume_vs_5d_avg_vol']>1) ]
runAll('G4', 'Full', '5dAvgVol>1', df)

df = merged_tweet_bowdf.loc[(merged_tweet_bowdf['no_of_positive_words']>=2) | (merged_tweet_bowdf['no_of_negative_words']>=2)]
runAll('G5', 'Full', 'NoOfImpWords>=2', df)

df = merged_tweet_bowdf.loc[(merged_tweet_bowdf['followers_count']>5000) & (merged_tweet_bowdf['hoursDiffFromParent']<2)]
runAll('G6', 'Full', 'Freshness<2 & Follower>5K', df)

df = merged_tweet_bowdf.loc[(merged_tweet_bowdf['no_of_positive_words']>=2) | (merged_tweet_bowdf['no_of_negative_words']>=2)]
df = df.loc[(df['volume_vs_5d_avg_vol']>1)]
runAll('G7', 'Full', '5dAvgVol>1 & NoOfImpWords>=2', df)


/home/kooud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kooud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kooud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [46]:
df = top_tweets
runAll('G1', 'TopTweets', 'None', df)

df = top_tweets.loc[(top_tweets['followers_count']>1000)]
runAll('G2', 'TopTweets', 'Follower>1K', df)

df = top_tweets.loc[(top_tweets['followers_count']>1000) & (top_tweets['hoursDiffFromParent']<2)]
runAll('G3', 'TopTweets', 'Freshness<2 & Follower>1K', df)

df = top_tweets.loc[(top_tweets['volume_vs_5d_avg_vol']>1) ]
runAll('G4', 'TopTweets', '5dAvgVol>1', df)

df = top_tweets.loc[(top_tweets['no_of_positive_words']>=2) | (top_tweets['no_of_negative_words']>=2)]
runAll('G5', 'TopTweets', 'NoOfImpWords>=2', df)

df = top_tweets.loc[(top_tweets['followers_count']>5000) & (top_tweets['hoursDiffFromParent']<2)]
runAll('G6', 'TopTweets', 'Freshness<2 & Follower>5K', df)

df = top_tweets.loc[(top_tweets['no_of_positive_words']>=2) | (top_tweets['no_of_negative_words']>=2)]
df = df.loc[(df['volume_vs_5d_avg_vol']>1)]
runAll('G7', 'TopTweets', '5dAvgVol>1 & NoOfImpWords>=2', df)

/home/kooud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/kooud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/kooud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [50]:
iteration_info.shape

(70, 21)

In [55]:
iteration_info.head(35)

,Group,Dataset,Filter,Model,TN,FP,FN,TP,Accuracy,Sensitivity,Specificity,Precision,Recall,F1,AUC,MCC,Long_gain_mean,Short_gain_mean,Test_size,Train_size,TotalAvgProfit
0,G1,Full,None,RF,2920,764,241,9546,0.925395,0.975375,0.792617,0.925897,0.975375,0.949993,0.883996,0.807793,1.547581,-1.389691,13471,31432,1.51053
1,G1,Full,None,GNB,3466,218,5420,4367,0.581471,0.446204,0.940825,0.952454,0.446204,0.607709,0.693515,0.364087,1.653015,-1.506540,13471,31432,1.55639
2,G1,Full,None,MNB,2308,1376,138,9649,0.887610,0.985900,0.626493,0.875193,0.985900,0.927254,0.806196,0.708104,1.555652,-1.372260,13471,31432,1.52235
3,G1,Full,None,BNB,3200,484,587,9200,0.920496,0.940022,0.868621,0.950021,0.940022,0.944995,0.904322,0.801801,1.565527,-1.415014,13471,31432,1.52321
4,G1,Full,None,XGB,3266,418,214,9573,0.953084,0.978134,0.886536,0.958162,0.978134,0.968045,0.932335,0.880524,1.547632,-1.384274,13471,31432,1.50543
5,G2,Full,Follower>1K,RF,788,333,72,2617,0.893701,0.973224,0.702944,0.887119,0.973224,0.928179,0.838084,0.737043,1.534166,-1.538105,3810,8890,1.53505
6,G2,Full,Follower>1K,GNB,950,171,1256,1433,0.625459,0.532912,0.847458,0.893392,0.532912,0.667598,0.690185,0.351074,1.498977,-1.583601,3810,8890,1.54797
7,G2,Full,Follower>1K,MNB,566,555,51,2638,0.840945,0.981034,0.504906,0.826182,0.981034,0.896974,0.742970,0.601089,1.526428,-1.350660,3810,8890,1.49796
8,G2,Full,Follower>1K,BNB,928,193,237,2452,0.887139,0.911863,0.827832,0.927032,0.911863,0.919385,0.869848,0.731603,1.542385,-1.511518,3810,8890,1.53295
9,G2,Full,Follower>1K,XGB,917,204,123,2566,0.914173,0.954258,0.818020,0.926354,0.954258,0.940099,0.886139,0.789978,1.520293,-1.562425,3810,8890,1.53179


In [53]:
# Add Additing info
iteration_info['Test_size'] = iteration_info['TN']+ iteration_info['FP'] + iteration_info['FN'] +iteration_info['TP']
iteration_info['Train_size'] = iteration_info['Test_size']*7/3
iteration_info['Train_size'] = iteration_info['Train_size'].astype('int32')
iteration_info['TotalAvgProfit'] = (iteration_info['Long_gain_mean']*(iteration_info['TP']+iteration_info['FP']) + iteration_info['Short_gain_mean']*(iteration_info['TN']+iteration_info['FN'])*-1)/(iteration_info['TN'] + iteration_info['TP'] + iteration_info['FN'] + iteration_info['FP'])

In [54]:
import pickle
# Save
with open('bowmodel_iteration_info.pickle', 'wb') as filehandle:
    pickle.dump(iteration_info, filehandle, protocol=pickle.HIGHEST_PROTOCOL)